[Reference](https://towardsdatascience.com/from-raw-data-to-web-app-deployment-with-atom-and-streamlit-d8df381aa19f)

# Set up


In [8]:
!pip install streamlit
!pip install atom-ml

     |████████████████████████████████| 9.2 MB 4.3 MB/s 
     |████████████████████████████████| 164 kB 42.4 MB/s 
     |████████████████████████████████| 180 kB 55.2 MB/s 
     |████████████████████████████████| 111 kB 61.3 MB/s 
     |████████████████████████████████| 76 kB 5.3 MB/s 
     |████████████████████████████████| 4.3 MB 50.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 126 kB 71.5 MB/s 
     |████████████████████████████████| 791 kB 61.9 MB/s 
     |████████████████████████████████| 374 kB 60.8 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=9a24f2d301ce32d2432a2c6e608d5cfc1b8140bdb7936d10c2540ea1e906920b
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

     |████████████████████████████████| 384 kB 5.2 MB/s 
     |████████████████████████████████| 11.3 MB 59.6 MB/s 
     |████████████████████████████████| 261 kB 70.0 MB/s 
     |████████████████████████████████| 305 kB 53.6 MB/s 
     |████████████████████████████████| 15.5 MB 47.9 MB/s 
     |████████████████████████████████| 100 kB 10.4 MB/s 
     |████████████████████████████████| 1.5 MB 54.1 MB/s 
     |████████████████████████████████| 87 kB 7.3 MB/s 
     |████████████████████████████████| 82 kB 435 kB/s 
     |████████████████████████████████| 159 kB 64.4 MB/s 
     |████████████████████████████████| 336 kB 64.1 MB/s 
     |████████████████████████████████| 41 kB 258 kB/s 
     |████████████████████████████████| 11.2 MB 48.7 MB/s 
     |████████████████████████████████| 564 kB 57.9 MB/s 
     |████████████████████████████████| 366 kB 57.8 MB/s 
     |████████████████████████████████| 470 kB 50.4 MB/s 
     |████████████████████████████████| 7.3 MB 23.7 MB/s 
     |████████████

In [1]:
import pandas as pd
import streamlit as st
from atom import ATOMClassifier
# Expand the web app across the whole screen
st.set_page_config(layout="wide")

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


# Pipeline menu


In [2]:
st.sidebar.title("Pipeline")

# Data cleaning options
st.sidebar.subheader("Data cleaning")
scale = st.sidebar.checkbox("Scale", False, "scale")
encode = st.sidebar.checkbox("Encode", False, "encode")
impute = st.sidebar.checkbox("Impute", False, "impute")

2021-12-26 11:46:41.303 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [3]:
# Model options
st.sidebar.subheader("Models")
models = {
    "gnb": st.sidebar.checkbox("Gaussian Naive Bayes", True, "gnb"),
    "rf": st.sidebar.checkbox("Random Forest", True, "rf"),
    "et": st.sidebar.checkbox("Extra-Trees", False, "et"),
    "xgb": st.sidebar.checkbox("XGBoost", False, "xgb"),
    "lgb": st.sidebar.checkbox("LightGBM", False, "lgb"),
}

# Data ingestion


In [4]:
st.header("Data")
data = st.file_uploader("Upload data:", type="csv")
# If a dataset is uploaded, show a preview
if data is not None:
    data = pd.read_csv(data)
    st.text("Data preview:")
    st.dataframe(data.head())

# Model training and evaluation


In [5]:
st.header("Results")

if st.sidebar.button("Run"):
    placeholder = st.empty()  # Empty to overwrite write statements
    placeholder.write("Initializing atom...")

    # Initialize atom
    atom = ATOMClassifier(data, verbose=2, random_state=1)

    if scale:
        placeholder.write("Scaling the data...")
        atom.scale()
    if encode:
        placeholder.write("Encoding the categorical features...")
        atom.encode(strategy="LeaveOneOut", max_onehot=10)
    if impute:
        placeholder.write("Imputing the missing values...")
        atom.impute(strat_num="median", strat_cat="most_frequent")
    
    placeholder.write("Fitting the models...")
    to_run = [key for key, value in models.items() if value]
    atom.run(models=to_run, metric="f1")
    
    # Display metric results
    placeholder.write(atom.evaluate())

    # Draw plots
    col1, col2 = st.beta_columns(2)
    col1.write(atom.plot_roc(title="ROC curve", display=None))
    col2.write(atom.plot_prc(title="PR curve", display=None))

else:
    st.write("No results yet. Click the run button!")